In [10]:
import pandas as pd
import numpy as np
from utils import *

In [11]:
# -- INVENTORY_LEVELS doeltabel --

# INVENTORY_LEVELS_ID (pk)
# INVENTORY_LEVELS_inventory_count
# PRODUCT_production_cost
# PRODUCT_margin
# ORDER_DETAILS_unit_cost
# ORDER_DETAILS_unit_price
# ORDER_DETAILS_unit_sale_price
# ORDER_DETAILS_quantity
# INVENTORY_LEVELS_turnover_rate* (calculateTurnoverRate(): turnover_rate = quantity / inventory_count)
# PRODUCT_product_number (fk uit ORDER_DETAILS)

In [12]:
go_sales = load_db('../source/go_sales.sqlite')
go_crm = load_db('../source/go_crm.sqlite')

In [13]:
inventory_levels = load_csv('../source/GO_SALES_INVENTORY_LEVELSData.csv', np.arange(0,4)).add_prefix('INVENTORY_LEVELS_')
order_header = load_table(go_sales, 'ORDER_HEADER').add_prefix('ORDER_HEADER_')
order_details = load_table(go_sales, 'ORDER_DETAILS').add_prefix('ORDER_DETAILS_')
sales_staff = load_table(go_sales, 'SALES_STAFF').add_prefix('SALES_STAFF_')
sales_branch = load_table(go_sales, 'SALES_BRANCH').add_prefix('SALES_BRANCH_')
product = load_table(go_sales, 'PRODUCT').add_prefix('PRODUCT_')

In [14]:
inventory_levels

,INVENTORY_LEVELS_INVENTORY_YEAR,INVENTORY_LEVELS_INVENTORY_MONTH,INVENTORY_LEVELS_PRODUCT_NUMBER,INVENTORY_LEVELS_INVENTORY_COUNT
0,2021,4,48,1932
1,2021,4,49,1400
2,2021,4,50,21705
3,2021,4,51,9710
4,2021,4,52,5616
...,...,...,...,...
3883,2022,12,111,1128
3884,2022,12,112,7810
3885,2022,12,113,3485
3886,2022,12,114,350


In [15]:
# Merge columns
# sales_target_data.product = order_details.product_number

product = product[['PRODUCT_PRODUCT_NUMBER', 'PRODUCT_PRODUCTION_COST', 'PRODUCT_MARGIN']]
product['PRODUCT_PRODUCT_NUMBER'] = product['PRODUCT_PRODUCT_NUMBER'].astype(int)
product['PRODUCT_MARGIN'] = product['PRODUCT_MARGIN'].astype(float)
product['PRODUCT_PRODUCTION_COST'] = product['PRODUCT_PRODUCTION_COST'].astype(float)

result = inventory_levels.merge(product, left_on='INVENTORY_LEVELS_PRODUCT_NUMBER', right_on='PRODUCT_PRODUCT_NUMBER')
result



,INVENTORY_LEVELS_INVENTORY_YEAR,INVENTORY_LEVELS_INVENTORY_MONTH,INVENTORY_LEVELS_PRODUCT_NUMBER,INVENTORY_LEVELS_INVENTORY_COUNT,PRODUCT_PRODUCT_NUMBER,PRODUCT_PRODUCTION_COST,PRODUCT_MARGIN
0,2021,4,48,1932,48,55.0,0.50
1,2021,4,49,1400,49,16.5,0.50
2,2021,4,50,21705,50,2.0,0.50
3,2021,4,51,9710,51,35.0,0.50
4,2021,4,52,5616,52,19.0,0.50
...,...,...,...,...,...,...,...
3883,2022,12,111,1128,111,81.8,0.55
3884,2022,12,112,7810,112,6.0,0.33
3885,2022,12,113,3485,113,6.0,0.50
3886,2022,12,114,350,114,80.0,0.60


In [16]:
# Add year_month date to order_detail by first joining order_header 
order = order_details.merge(order_header, left_on='ORDER_DETAILS_ORDER_NUMBER', right_on='ORDER_HEADER_ORDER_NUMBER')
order['ORDER_HEADER_ORDER_DATE'] = pd.to_datetime(order['ORDER_HEADER_ORDER_DATE'])
order['YEAR_MONTH_YEAR'] = order['ORDER_HEADER_ORDER_DATE'].dt.year.astype(int)
order['YEAR_MONTH_MONTH'] = order['ORDER_HEADER_ORDER_DATE'].dt.month.astype(int)
order['ORDER_DETAILS_PRODUCT_NUMBER'] = order['ORDER_DETAILS_PRODUCT_NUMBER'].astype(int)
order = order[['ORDER_DETAILS_PRODUCT_NUMBER', 'ORDER_DETAILS_QUANTITY', 'YEAR_MONTH_YEAR', 'YEAR_MONTH_MONTH']]
order = order.groupby(['ORDER_DETAILS_PRODUCT_NUMBER', 'YEAR_MONTH_YEAR', 'YEAR_MONTH_MONTH']).sum().reset_index()
result.dtypes


INVENTORY_LEVELS_INVENTORY_YEAR       int64
INVENTORY_LEVELS_INVENTORY_MONTH      int64
INVENTORY_LEVELS_PRODUCT_NUMBER       int64
INVENTORY_LEVELS_INVENTORY_COUNT      int64
PRODUCT_PRODUCT_NUMBER                int32
PRODUCT_PRODUCTION_COST             float64
PRODUCT_MARGIN                      float64
dtype: object

In [17]:
order.dtypes

ORDER_DETAILS_PRODUCT_NUMBER    int32
YEAR_MONTH_YEAR                 int32
YEAR_MONTH_MONTH                int32
ORDER_DETAILS_QUANTITY          int64
dtype: object

In [18]:
# Merge order with inventory_levels, year, month and product_number
result = result.merge(order, left_on=['INVENTORY_LEVELS_PRODUCT_NUMBER', 'INVENTORY_LEVELS_INVENTORY_YEAR', 'INVENTORY_LEVELS_INVENTORY_MONTH'], right_on=['ORDER_DETAILS_PRODUCT_NUMBER', 'YEAR_MONTH_YEAR', 'YEAR_MONTH_MONTH'], how='left')

# Calculate turnover_rate
result['INVENTORY_LEVELS_TURNOVER_RATE'] = result['ORDER_DETAILS_QUANTITY'] / result['INVENTORY_LEVELS_INVENTORY_COUNT']

# Remove duplicate columns
result = result.drop(columns=['ORDER_DETAILS_PRODUCT_NUMBER', 'INVENTORY_LEVELS_INVENTORY_YEAR', 'INVENTORY_LEVELS_INVENTORY_MONTH', 'INVENTORY_LEVELS_PRODUCT_NUMBER', 'ORDER_DETAILS_QUANTITY'])
result

,INVENTORY_LEVELS_INVENTORY_COUNT,PRODUCT_PRODUCT_NUMBER,PRODUCT_PRODUCTION_COST,PRODUCT_MARGIN,YEAR_MONTH_YEAR,YEAR_MONTH_MONTH,INVENTORY_LEVELS_TURNOVER_RATE
0,1932,48,55.0,0.50,2021.0,4.0,0.069358
1,1400,49,16.5,0.50,2021.0,4.0,0.171429
2,21705,50,2.0,0.50,2021.0,4.0,0.154250
3,9710,51,35.0,0.50,2021.0,4.0,0.119053
4,5616,52,19.0,0.50,2021.0,4.0,0.182692
...,...,...,...,...,...,...,...
3883,1128,111,81.8,0.55,2022.0,12.0,0.106383
3884,7810,112,6.0,0.33,2022.0,12.0,0.046863
3885,3485,113,6.0,0.50,2022.0,12.0,0.099283
3886,350,114,80.0,0.60,2022.0,12.0,0.142857
